In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import bp_pas.params as params
import bp_pas.corpus as corpus
import bp_pas.stats as stats
from bp_pas.eval import evaluate

import sys
import gflags
import tensorflow as tf
import numpy as np

In [ ]:
params.load_defaults()
flags = gflags.FLAGS
#arg_str = '' #'--train_data single.utf8 --dev_data single.utf8 --test_data single.utf8'
arg_str = '--max_train_instances 1' #'--train_data single.utf8 --dev_data single.utf8 --test_data single.utf8'
args = [''] + arg_str.split(' ')
argv = flags(args)

In [ ]:
# tmp params to add to flags
unk_threshold = 10
unk_token = '<UNK>'
arg_embedding_size = 32
pred_embedding_size = 8
num_training_instances = 1


In [ ]:
# Load data
print(flags.train_data)
ntc = corpus.NTCLoader()
train_data = ntc.load_corpus(flags.train_data, flags.max_train_instances)
test_data  = ntc.load_corpus(flags.test_data, flags.max_train_instances)
dev_data   = ntc.load_corpus(flags.dev_data, flags.max_train_instances)
print('{} train sentences.'.format(len(train_data)))
print('{} test sentences.'.format(len(test_data)))
print('{} dev sentences.'.format(len(dev_data)))
stats.corpus_statistics(train_data)
stats.show_case_dist(train_data)

In [ ]:
from collections import Counter
class Vocab:
    
    def __init__(self, init_words=[], unk_token = '<UNK>', unk_threshold=10):
        self.word2int = {}
        self.int2word = []
        self.frozen = False
        self.add(unk_token)
        if len(init_words) > 0:
            self.build(init_words, unk_threshold)

    def add(self, elem):
        assert not self.frozen
        if elem not in self.word2int:
            self.int2word.append(elem)
            self.word2int[elem] = len(self) - 1
    
    def build(self, words, unk_threshold=10):
        counter = Counter()
        for w in words:
            counter[w] += 1
        for w,c in counter.items():
            if c > unk_threshold:
                self.add(w)
    
    def freeze(self):
        self.frozen = True

    def index(self, elem):
        if self.frozen and elem not in self.word2int:
            return 0
        else:
            assert elem in self.word2int
            return self.word2int[elem]
    
    def word(self, index):
        assert index < self.int2word
        return self.int2word[index]
    
    def __len__(self):
        return len(self.int2word)


In [ ]:
# Setup vocabulary
all_sents = train_data[0] + test_data[0] + dev_data[0]


print('Collecting all tokens...')
word_tokens = list(word.form
                   for sent in all_sents
                   for word in sent)
print('Total tokens: {}\n'.format(len(word_tokens)))

print('Building (argument) vocabulary...')
arg_vocab = Vocab(init_words=word_tokens, 
                  unk_token=unk_token, 
                  unk_threshold=unk_threshold)  #[unk_token] + get_vocab(word_tokens)
arg_vocab.freeze()
arg_vocabulary_size = len(arg_vocab)
print('Vocabulary size: {}\n'.format(arg_vocabulary_size))

print('Collecting predicates...')
pred_vocab = Vocab(init_words=[word.form
                               for sent in all_sents
                               for word in sent if word.is_prd])
pred_vocab.freeze()
pred_vocabulary_size = len(pred_vocab)
print('Number of predicates: {}\n'.format(pred_vocabulary_size))

print('Collecting argument types...')
arg_types = ['NIL'] + list(set([arg.arg_type 
                      for sent in all_sents 
                      for pas in sent.pas
                      for arg in pas.args]))
#for sent in train_data[0]:
#    for pas in sent.pas:
#        print(pas.pred.word_index)
#        for arg in pas.args:
#            print(arg)
print('Arg types: ', arg_types)
num_types = len(arg_types)

In [ ]:
# Convert data structures into numpy ndarrays for placeholders
def sent2nump(sent):
    sent_ids = [arg_vocab.index(w.form) for w in sent]
    pases = [w.form for w in sent if w.is_prd]
    pred_ids = [pred_vocab.index(w.form) for w in sent if w.is_prd]
#    print(pas_ids)
#    print(pases)
    label_mats = []
    for pas in sent.pas:
        lm = label_matrix(pas, arg_types, len(sent))
        label_mats.append(lm)
    lm = np.concatenate(label_mats, axis=1) #.shape
    return [sent_ids], [pred_ids], lm
#        break

def label_matrix(pas, arg_types, sent_len):
    zeros = np.zeros((len(arg_types), sent_len))
    for arg in pas.args:
        zeros[arg_types.index(arg.arg_type), arg.word_index] = 1.0
    for j in range(sent_len):
        found = False
        for i in range(1, len(arg_types)):
            if zeros[i, j] == 1.0:
                found = True
        if not found:
            zeros[0,j] = 1.0
    return zeros







#def pas2matrix(pas):
    

def sent2ints(sent):
    sent_ids = [vocab.index(w.form) for w in sent]
    pas_ids = []
    return sent_ids, pas_ids

#def sents2batch(sents, vocab, batch_size):
#    for sent in sents:
#
#    int_sents = [ for sent in train_data[0]]
#    return np.array([int_sents[0]]), np.array([int_sents[0]])


#sents, labels = sents2batch(train_data, vocab, batch_size=1)
#print(batched_sent_dicts.shape)

d = train_data[0][0]
#print(len(d))
#print(' '.join([w.form for w in d.words]))
#for w in d:
#    if w.is_prd:
#        print(w, '\t', w.arg_types, '\t  ', w.arg_indices)
#sent2nump(d)

#GA_INDEX = 0
#O_INDEX = 1
#NI_INDEX = 2

# pred id 10 has NI and GA
# pred id 25 has GA and O
# pred id 31 has GA

train_dicts = [sent2nump(td) for td in train_data[0][1:5]]
#sent_ids, pred_ids = sent2nump(train_data[0][0])
#print(sent_ids)
#print(pred_ids)

In [ ]:
# Integerize the sentences
print(len(int_sents))
print(int_sents[0])

In [ ]:
# Setup the context embedding
tf.reset_default_graph()

# Construct embedding matrix for contex/arguments
arg_embeddings = tf.Variable(tf.random_uniform([arg_vocabulary_size, arg_embedding_size], -1.0, 1.0, dtype=tf.float64))

# Construct embedding matrix for predicates
pred_embeddings = tf.Variable(tf.random_uniform([pred_vocabulary_size, num_types * pred_embedding_size], -1.0, 1.0, dtype=tf.float64))

# Setup placeholders
batch_size = 1
sent_placeholder = tf.placeholder(tf.int32, shape=(batch_size, None))
pred_placeholder = tf.placeholder(tf.int32, shape=(batch_size, None))
gold_placeholder = tf.placeholder(tf.float64, shape=(num_types, None))

def context_embeddings(sent, arg_embeddings, output_dim):
    # Shape [batch_size, max_sent_len, emb_dim]
    embed_rep = tf.gather(arg_embeddings, sent)
    # List of length max_sent_len, comprising [batch_size, emb_dim] tensors
#    X = tf.unstack(embed_rep, axis=1, num=72)
#    return X
    X = embed_rep
#    return X
    fw_cell = tf.contrib.rnn.LSTMCell(num_units=output_dim/2, state_is_tuple=True)
    bw_cell = tf.contrib.rnn.LSTMCell(num_units=output_dim/2, state_is_tuple=True)
    outputs, states  = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=fw_cell,
        cell_bw=bw_cell,
        dtype=tf.float64,
#        sequence_length=X_lengths,
        inputs=X)
    output_fw, output_bw = outputs
    states_fw, states_bw = states
    final_rep = tf.concat(outputs, 2)
    return final_rep


def pred_full_scoring(preds, pred_embeddings, context_mat):
    # Reshape the context mat to remove the batch dim
    context_tensor = tf.reshape(context_mat, shape=[-1, tf.shape(context_mat)[-1]])

    # Shape [batch_size, num_tokens, pred_embed_dim]
    # actually ,remove batch for now
    pred_mat = tf.gather(pred_embeddings, preds)
    pred_mat = tf.squeeze(pred_mat)

    # Split the pred embeddings into individual pred tensors
    score_tensor = tf.map_fn(lambda x: scoring(x, context_tensor), pred_mat)
    
    # Reshape score tensor for loss tensor
    score_tensor = tf.transpose(tf.reshape(score_tensor, shape=[-1, tf.shape(score_tensor)[-1]]))
    return score_tensor


def scoring(pred_tensor, context_tensor):
#    pred_tensor = tf.reshape(pred_tensor, shape=[1, -1, 4])
    pred_tensor = tf.reshape(pred_tensor, shape=[tf.shape(context_tensor)[-1], -1])
    return tf.matmul(context_tensor, pred_tensor)

#def pred_single_scoring(single_pred_set, single_context, num_slots, num_preds):
#    return single_pred_set
#    single_pred = tf.unstack(single_pred_set, num=num_preds)[0]
#    return tf.transpose(single_pred)
#    return tf.matmul(single_context, tf.transpose(single_pred))

def srl_loss(gold, preds, pred_embeddings, context_mat):
    prediction = pred_full_scoring(pred_placeholder, pred_embeddings, context_rep)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=tf.transpose(prediction), 
                                                   labels=tf.transpose(gold))
    return loss

context_rep = context_embeddings(sent_placeholder, arg_embeddings, output_dim=8)
pred_scores = pred_full_scoring(pred_placeholder, pred_embeddings, context_rep)


In [ ]:
# Decoding method
from bp_pas.ling.pas import PAS, Predicate, Argument
from bp_pas.ling.sent import Sentence

def decode(sess, instance):
    sent_ids, pred_ids, gold_labels = sent2nump(instance)
    scores = sess.run(pred_scores, feed_dict = {
            sent_placeholder: sent_ids,
            pred_placeholder: pred_ids,
            gold_placeholder: gold_labels        
    })
    pred_indices = [i for i, w in enumerate(instance.words) if w.is_prd]
    num_preds = len(pred_indices)
    scored_by_pred = np.split(scores, indices_or_sections=num_preds, axis=1)
    pases = []
    for pred_id, scored_mat in zip(pred_indices, scored_by_pred):
#        print(pred_id)
        pred = Predicate(word_index = pred_id,
                         word_form = instance.words[pred_id].form)
        args = []
        for slot, slot_id in zip(arg_types, range(num_types)):
            if slot != 'NIL':
                max_index = np.argmax(scored_mat[slot_id])
                args.append(Argument(word_index=max_index,
                                     word_form=instance.words[max_index].form,
                                     arg_type=slot))
#                print('\t', max_index)
        pases.append(PAS(pred, args))
    return Sentence(instance.words, pases)

decode(sess, train_data[0][0])

In [ ]:
from bp_pas.eval import evaluate

sess = tf.Session()
sess.run(tf.global_variables_initializer())

opt = tf.train.GradientDescentOptimizer(learning_rate=0.001)
loss = srl_loss(gold_placeholder, pred_placeholder, pred_embeddings, context_rep)
opt_op = opt.minimize(loss)

num_epochs = 10000
for epoch in range(num_epochs):
    epoch_loss = 0
    for train_dict in train_dicts:
        sent_ids, pred_ids, gold_labels = train_dict
#        print(gold_labels.shape)
        _, closs = sess.run([opt_op, loss], feed_dict = {
            sent_placeholder: sent_ids,
            pred_placeholder: pred_ids,
            gold_placeholder: gold_labels
        })
        epoch_loss += closs.mean()
    if epoch % 100 == 0:
        decoded = [pas for pas in decode(sess, sent).pas for sent in train_data[0][:5]]
        gold = [pas for pas in sent.pas for sent in train_data[0][:5]]
        evaluate(decoded, gold)
        print('{0:>4}: {1:.2f}'.format(epoch, epoch_loss))
    

#feed_dict = {sent_placeholder: sent_ids, pred_placeholder: pred_ids}
#context_out = sess.run(context_rep, feed_dict=feed_dict)
#print(context_out.shape)
#out = sess.run(pred_scores, feed_dict=feed_dict)
#print(out)
#print(out.shape)
#print(out[0].shape)

# 1, 9, 8

#want 1, 9, 1



# context mat is 71, 8

# pred mat is 8, 36

# after mult is 71 x 36


In [ ]:
# Decoding method
from bp_pas.ling.pas import PAS, Predicate, Argument
from bp_pas.ling.sent import Sentence

def decode(sess, instance):
    sent_ids, pred_ids, gold_labels = sent2nump(instance)
    scores = sess.run(pred_scores, feed_dict = {
            sent_placeholder: sent_ids,
            pred_placeholder: pred_ids,
            gold_placeholder: gold_labels        
    })
    pred_indices = [i for i, w in enumerate(instance.words) if w.is_prd]
    num_preds = len(pred_indices)
    scored_by_pred = np.split(scores, indices_or_sections=num_preds, axis=1)
    pases = []
    for pred_id, scored_mat in zip(pred_indices, scored_by_pred):
#        print(pred_id)
        pred = Predicate(word_index = pred_id,
                         word_form = instance.words[pred_id].form)
        args = []
        for slot, slot_id in zip(arg_types, range(num_types)):
            if slot != 'NIL':
                max_index = np.argmax(scored_mat[slot_id])
                args.append(Argument(word_index=max_index,
                                     word_form=instance.words[max_index].form,
                                     arg_type=slot))
#                print('\t', max_index)
        pases.append(PAS(pred, args))
    return Sentence(instance.words, pases)

decode(sess, train_data[0][0])